# Analysing the redundancies of User Stories benefits

In [4]:
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.request_handler import send_requierment_to_chatgpt, StoppedAnswerException
from support_functions.time_recorder import TimeRecorder
from support_functions.save_data import save_to_json_persistent
from itertools import combinations
import pandas as pd
import json
import tiktoken
import os
import locale
# import multiprocessing
# from multiprocessing import Lock, Process, Queue, Manager, Value
# import time
# import ctypes

In [5]:
if __name__ == "__main__":
    pass
locale.setlocale(locale.LC_ALL, 'de_DE') 
MODEL_VERSION_NAME = os.getenv('MODEL_VERSION')
THREADING = bool(os.getenv('THREADING'))
print(MODEL_VERSION_NAME)

gpt-3.5-turbo


### Load all User Stories from the different packages

In [6]:
datasets: dict[str, list] = loading()

### Datatransformation to panda dataframes and numpy arrarys. 

In [7]:
def transform_data_id_text(data: list):
    rows = []
    for entry in data:
        row = {
            'ID': entry['id'],
            'User Story': entry['text']
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [8]:
# n(n-1) / 2
def transform_pairwise(df: pd.DataFrame):
    rows = []
    for i, j in combinations(range(df.shape[0]), 2):
        row = {
            'First ID': df.iloc[i, 0],
            'First User Story': df.iloc[i, 1],
            'Second ID': df.iloc[j, 0],
            'Second User Story': df.iloc[j, 1]
        }
        rows.append(row)
    return pd.DataFrame(rows)

## Request definition for User Stories

In [9]:
definition_main_part: str = "The main part of a User Story describes the core action that a persona wishes to accomplish. For example, the main part of the user story 'As a UI designer, I want to begin user testing, so that I can validate stakeholder UI improvement requests' is 'As a UI designer, I want to begin user testing'. This part specifies the main functionality or feature the story aims to deliver."
definition_benefit: str = "A benefit in a User Story refers to the positive outcome or advantage that a user or stakeholder gains from the functionality described in the User Story. It directly addresses user needs and enhances the value of the software product, providing a clear motive for the user action, like easing a task or improving efficiency. We use the example from before which was 'As a UI designer, I want to begin user testing, so that I can validate stakeholder UI improvement requests'. Here we determine the benefit part, which is 'so that I can validate stakeholder UI improvement request'. Hence, a benefit starts with the phrase 'so that'."
definition_general_redundancy: str = "Redundancy in a User Story refers to the unnecessary repetition or duplication of information within the story's description. Such redundancies can cloud the core purpose or value of the feature, leading to potential confusion or misinterpretation by the development team. It's crucial to remove redundancies to maintain clarity and focus in development."
definition_main_part_redundancy: str = "A main part redundancy in User Stories refers to unnecessary duplication in the description of the action or functionality that the persona aims to achieve, excluding persona repetitions and benefits. This type of redundancy occurs when similar actions or functionalities are described in multiple stories without adding distinct value or clarity, potentially leading to confusion or inefficiencies in the development process. Thus, a main part redundancy occurs when a pair of two User Stories have a redundant main part."
definition_benefit_redundancy: str = "Benefit redundancy in User Stories occurs when multiple stories repeatedly emphasize the same positive outcomes of similar software features. This duplication can lead to inefficiencies as the same advantages are stated multiple times, potentially diluting the impact and clarity of user benefits in the product documentation. Thus, a benefit redundancy occurs when a pair of two User Stories have a redundant benefit part."

In [10]:
### Maybe add a field for explanation
### Define that the fist entry of the referenceToOriginalText has to be from the first user story and the second from the second
### Add a describtion why it is redundant

definition_json_format_pair_user_stories: str = '''
{
    "relatedStories": [1, 3],
    "redundantMainPart": true, 
    "mainPartRedundancies": [
        {
            "reasonDescribtion": "The redundancy arises from the repetition of the concept of user verification expressed in slightly different ways for emphasis or to cater to different audiences.",
            "referenceToOriginalText": ["As a User, I want to verify myself", "As an application User, I want to verify that I am who I am"]
        },
        {
            "reasonDescribtion": "The redundancy here stems from the repetition of the testing action and the tested entity, expressed in slightly different ways, likely for emphasis or to accommodate different perspectives",
            "referenceToOriginalText": ["As a software tester, I want to begin user testing", "As a tester, I want to begin testing the software"]
        }
        // Additional main part redundancies can be added here
    ],
    "redundantBenefit": true, 
    "benefitRedundancies": [
        {
            "reasonDescribtion": "The redundancy in these sentences lies in the repetition of the action ("login") and the destination ("webpage" and "website"), which convey the same idea using slightly different wording.",
            "referenceToOriginalText": ["I can login into the webpage.", "I could login into the website"]
        },
        {
            "reasonDescribtion": "The redundancy occurs due to the repetition of the action ('print') expressed in slightly different ways, conveying the same meaning of initiating the printing process.",
            "referenceToOriginalText": ["I can print a document", "I can give the order to print"]
        }
        // Additional benefit redundancies can be added here
    ]
}
'''

### Request definition to ChatGPT

In [11]:
#Collection of messages -> In future it can be used to create a context aware system
message: list[dict] = []

#Defining initial user request
user_inital_message: dict = {
  "role": "user",
  "content": "Act as a Requirements Engineer focused on identifying redundancies. Please review pairs of two User Stories and pinpoint any unnecessary duplications that obscure clarity or add no distinct value."
}
message.append(user_inital_message)

# This was generated by ChatGPT thus it is marked as assistent
system_is_r_eng = ("As a requirements engineer in agile development, it is my responsibility to review user stories for redundancies. My goal is to identify and report any overlapping or duplicate requirements." 
" By carefully analysing the user stories in depth, I ensure that each requirement is necessary and contributes uniquely, increasing the coherence of the product.")
assistent_initial_message: str = {"role": "system", "content": system_is_r_eng}
message.append(assistent_initial_message)

#Defining the definition of a main part and benefit redundancy
user_defining_benefit = {"role": "user", "content": "Please, analyse redundancies in the main part and benefit of a pair of two given User Stories"
    " Note that a User Story may include multiple redundancies in the main part as well as the benefit. The redundancies of the main part and benefit are disjoint sets."
    " Hence, a main part can be redundant while a benefit is not and vice versa. However, in some cases the main part and the benefit can be at the same time redundant, but they do not depend on each other and therefore they are independent redundant." 
    " The definition of the main part is: "
    f"\"{definition_main_part}\" and the definition of a benefit is as follows: \"{definition_benefit}\""}
message.append(user_defining_benefit)

# This was generated by ChatGPT thus it is marked as assistent
system_is_r_eng_defintion_aware: str = ("I will analyse redundancies in the main parts and benefits of two User Stories. Each story might include multiple redundancies."
" The main part typically describes the desired functionality by the persona, while the benefit details the positive outcomes from the functionality.")
message.append({"role":"system","content":system_is_r_eng_defintion_aware})

#Defining what a redundancy is
user_defining_redundancy: dict = {"role": "user", "content":f"Please analyse the User Stories considering the general definition of redundancy: {definition_general_redundancy} "
    f"Additionally, take into account the specific redundancy in the main part: \"{definition_main_part_redundancy}\" "
    f"and the redundancy in benefits: \"{definition_benefit_redundancy}\" Can you do that?"}
message.append(user_defining_redundancy)

# This was generated by ChatGPT thus it is marked as assistant
system_is_r_eng_defintion_aware_redundancy: str = "I'll review the User Stories for redundancies in main parts and benefits, using the specified definitions. Ready to proceed?"
message.append({"role":"system", "content": system_is_r_eng_defintion_aware_redundancy})

#Going to the formating of the output
user_defining_return_json_format = {"role":"user", "content": f"Before we proceed, consider the following JSON output format: {definition_json_format_pair_user_stories}."
                                    +     " This JSON structure organizes information about redundancies in User Stories, focusing on both the main parts and the benefits."
                                    +     " Each section includes descriptions of the redundancies and specific text references to illustrate where these redundancies occur within the stories."
                                    + "1.) The field 'relatedStories' is an integer value. The value are the ids of the User Stories and this field is also mandatory. \n" 
                                    + "2.) The field 'redundantMainPart' can be true or false. true indicates that a pair of User Stories has at least one main part redundancy and otherwise false. It is a mandatory field. \n" 
                                    + "3.) The array 'mainPartRedundancies' contains no, one or more objects, each representing a specific redundancy found within the main parts of User Stories."
                                    +     " When no main part redundancy is found, it contains no objects and thus it is empty."
                                    +     " In the case only one main part redundancy is found, it contains one entry."
                                    +     " The last case is that multiple main part redundancies are found. Then it contains more than one entry."
                                    +     " This array is mandatory to be filled when the field 'redundantMainPart' is true. \n" 
                                    + "3.1.) The field 'redundancyDescription' should contain a brief description, typically a sentence or two, explaining the reason for the main part redundancy. \n"
                                    + "3.2.) The array 'referenceToOriginalText' holds the exact text from the User Story pairs that demonstrates the redundancy.  Thus, the array has two entries."
                                    +     " The first entry is from the first user story and the second entry is from the user story. \n"
                                    + "4.) The field 'redundantbenefit' can be true or false. true indicates that a pair of User Stories has at least one benefit redundancy and otherwise false. It is a mandatory field. \n"
                                    + "5.) The array 'benefitRedundancies' is similar to 'mainPartRedundancies' but contain not the same content."
                                    +     " The main part and benefit redundancies are disjoint, therefore a benefit redundancy can exist while a main part redundancy does not exist or exists."
                                    +     " This array details redundancies found in the benefits described by the User Story pairs. This array is mandatory to be filled when the field 'redundantBenefit' is true.  \n" 
                                    + "5.1) The field 'redundancyDescription' should contain a brief description, typically a sentence or two, explaining the reason for the benefit redundancy. \n"
                                    + "5.2.) The array 'referenceToOriginalText' lists the text from the stories that highlight the redundant benefits, helping to pinpoint where exactly these redundancies occur."
                                    +     "Thus, the array has two entries. The first entry is from the first user story and the second entry is from the user story."
                                    +     " When no main part redundancy is found, it contains no objects and thus it is empty."
                                    +     " In the case only one main part redundancy is found, it contains one entry."
                                    +     " The last case is that multiple main part redundancies are found. Then it contains more than one entry."
                                    +     " This array is mandatory to be filled when the field 'redundantMainPart' is true. \n"                                 
                                    + "Are you ready to analyse redundancies in User Stories, focusing on main parts and benefits, and return the findings in JSON format?"}
message.append(user_defining_return_json_format)

#Defining that ChatGPT can do it
message.append({"role":"system", "content": "I've noted the JSON output format specified. I am ready to analyse redundancies in User Stories, focusing on the main parts and benefits. Shall we begin?"})


In [12]:
if True:
    message_text: str = ""
    for key in message:
        message_text += key["content"]
        print(key["content"])
    enc = tiktoken.get_encoding('cl100k_base')
    token_size = enc.encode(message_text)
    print("-" * 3 + "Token" + "-" * 3)
    print(token_size)
    print("-" * 3 + "Sum of Tokens" + "-" * 3)
    # The output is not correct as the result from the online pages differs https://platform.openai.com/tokenizer
    print("The total token sum is: " + locale.format_string("%d", sum(token_size), grouping=True))
    

Act as a Requirements Engineer focused on identifying redundancies. Please review pairs of two User Stories and pinpoint any unnecessary duplications that obscure clarity or add no distinct value.
As a requirements engineer in agile development, it is my responsibility to review user stories for redundancies. My goal is to identify and report any overlapping or duplicate requirements. By carefully analysing the user stories in depth, I ensure that each requirement is necessary and contributes uniquely, increasing the coherence of the product.
Please, analyse redundancies in the main part and benefit of a pair of two given User Stories Note that a User Story may include multiple redundancies in the main part as well as the benefit. The redundancies of the main part and benefit are disjoint sets. Hence, a main part can be redundant while a benefit is not and vice versa. However, in some cases the main part and the benefit can be at the same time redundant, but they do not depend on each 

## Data Processing Definition Pipline for User Stories

In [13]:
def templat_request_two_user_stories(current_message: list[dict], user_story_one: str, user_story_two: str, user_story_one_id: str, user_story_two_id: str,):
    request: dict = {"role":"user", "content":"Yes. Please, process the following pair of user stories:\n"
    f"id: {user_story_one_id}, describtion:  {user_story_one}\n"
    f"id: {user_story_two_id}, describtion: {user_story_two}"}
    current_message.append(request)

In [14]:
def process_user_stories(pairs: pd.DataFrame, results: list, exceptions_during_processing: list) -> None:   
    time_recorder: TimeRecorder = None
    for idx in range(len(pairs)):
        current_message: list[dict] = message.copy()
        templat_request_two_user_stories(current_message, pairs.iat[idx, 1], pairs.iat[idx, 3], str(pairs.iat[idx, 0]), str(pairs.iat[idx, 2]))
        try:
            time_recorder = TimeRecorder()
            resonse = send_requierment_to_chatgpt(current_message, time_recorder)
            json_object = json.loads(resonse)
            json_object = {'elipsedTimeNs': time_recorder.nanoseconds, **json_object}
            results.append(json_object)
        except StoppedAnswerException:  # Handle StoppedAnswerException
            exceptions_during_processing_data = {"Reason" : "Not stopped exception from ChatGPT Endpoint", "UID1" : str(pairs.iat[idx, 0]),  "UID2" : str(pairs.iat[idx, 2])}
            exceptions_during_processing.append(json.loads(exceptions_during_processing_data))
        except ValueError as e:  # Handle ValueError
            exceptions_during_processing_data = {"Reason" : "ValueError", "UID1" : str(pairs.iat[idx, 0]),  "UID2" : str(pairs.iat[idx, 2])}
            exceptions_during_processing.append(json.loads(exceptions_during_processing_data))

In [15]:
# GLOBAL_SAVE_LOCK = Lock
# WAIT_FOR_REQUESTS_FINISHED: int = 3 #in seconds

In [16]:
# def save_to_json_persistent_thread_save(folder_name, key, results_thread_save, exceptions_thread_save, lock, shared_countdown_flag):
#     #Sleeps here until all requests of a type are done
#     while shared_countdown_flag != 0:
#         time.sleep(WAIT_FOR_REQUESTS_FINISHED)
    
#     results_collection: dict = {}
#     results_collection[key] = list(results_thread_save)
#     if len(list(exceptions_thread_save)) != 0: results_collection[key.value.decode('utf-8'), 'Exceptions'] = list(exceptions_thread_save)
#     with lock:
#         save_to_json_persistent(folder_name.value.decode('utf-8'), results_collection)

In [17]:
# def manage_request(q_messages, results, exceptions_during_processing, shared_countdown_flag) -> None:
#     time_recorder: TimeRecorder = None
#     while not q_messages.empty():
#         d: dict = dict(q_messages.get())
#         usid1, usid2 = str(list(dict(q_messages.get()).keys())[0]).split(";")
#         print(usid1)
#         print(usid2)
#         local_messages = list(dict(q_messages.get()).values())[0]
#         try:
#             time_recorder = TimeRecorder()
#             resonse = send_requierment_to_chatgpt(local_messages, time_recorder)
#             json_object = json.loads(resonse)
#             json_object = {'elipsedTimeNs': time_recorder.nanoseconds, **json_object}
#             results.append(json_object)
#         except StoppedAnswerException:  # Handle StoppedAnswerException
#             exceptions_during_processing_data = {"Reason" : "Not stopped exception from ChatGPT Endpoint", "UID1" : usid1,  "UID2" : usid2}
#             exceptions_during_processing.append(json.loads(exceptions_during_processing_data))
#         except ValueError as e:  # Handle ValueError
#             exceptions_during_processing_data = {"Reason" : "ValueError", "UID1" : usid1,  "UID2" : usid2}
#             exceptions_during_processing.append(json.loads(exceptions_during_processing_data))
#         with shared_countdown_flag.get_lock():
#             shared_countdown_flag = shared_countdown_flag - 1

In [18]:
# def process_requests_parallel(pairs: pd.DataFrame, folder_name: str, key: str):    
#     with Manager() as manager:
        
#         #Data types for threading
#         requests_to_accomplish = Queue()
#         processes: list = []
#         results_thread_save = manager.list()
#         exceptions_thread_save = manager.list()
        
#         elements: int = 0
#         current_message: list[dict] = []
#         for idx in range(len(pairs)):
#             current_message = message.copy()
#             templat_request_two_user_stories(current_message, pairs.iat[idx, 1], pairs.iat[idx, 3], str(pairs.iat[idx, 0]), str(pairs.iat[idx, 2]))
#             requests_to_accomplish.put({pairs.iat[idx, 1] + ";" + pairs.iat[idx, 3] : current_message})
#             elements += 1
        
#         shared_countdown_flag = Value('i', elements)
        
#         for _ in range(os.cpu_count() * 2):
#             process = Process(target=manage_request, args=(requests_to_accomplish, results_thread_save, exceptions_thread_save, shared_countdown_flag))
#             processes.append(process)
#             process.start()
        
#         for process in processes:
#             process.join()
        
        # shared_folder_name = multiprocessing.Array(ctypes.c_char, len(folder_name) + 1)
        # shared_folder_name.value = folder_name.encode('utf-8')
        # shared_key = multiprocessing.Array(ctypes.c_char, len(key) + 1)
        # shared_key.value = key.encode('utf-8')

        # process = Process(target=save_to_json_persistent_thread_save, args=(shared_folder_name, shared_key, results_thread_save, exceptions_thread_save, GLOBAL_SAVE_LOCK, shared_countdown_flag))
        # process.start
        # process.join   
        # print("Reached End")
        

## Data processing for: G02 federal funding

In [98]:
id_g02: str = "#G02#"
data_g02: list = datasets[id_g02]
df_g02: pd.DataFrame = transform_data_id_text(data_g02)
df_g02_pairs = transform_pairwise(df_g02)

In [99]:
display(df_g02)

,ID,User Story
0,215,"As a Data user, I want to have the 12-19-2017 ..."
1,216,"As a UI designer, I want to redesign the Resou..."
2,217,"As a UI designer, I want to report to the Agen..."
3,218,"As a UI designer, I want to move on to round 2..."
4,220,"As a UI designer, I want to move on to round 3..."
...,...,...
90,308,"As a FABS user, I want to have my validations ..."
91,309,"As a FABS user, I want to see correct status l..."
92,310,"As an agency user, I want to know when the sub..."
93,311,"As an agency user, I want a landing page to na..."


In [100]:
display(df_g02_pairs)

,First ID,First User Story,Second ID,Second User Story
0,215,"As a Data user, I want to have the 12-19-2017 ...",216,"As a UI designer, I want to redesign the Resou..."
1,215,"As a Data user, I want to have the 12-19-2017 ...",217,"As a UI designer, I want to report to the Agen..."
2,215,"As a Data user, I want to have the 12-19-2017 ...",218,"As a UI designer, I want to move on to round 2..."
3,215,"As a Data user, I want to have the 12-19-2017 ...",220,"As a UI designer, I want to move on to round 3..."
4,215,"As a Data user, I want to have the 12-19-2017 ...",221,"As a Developer , I want to be able to log bett..."
...,...,...,...,...
4460,309,"As a FABS user, I want to see correct status l...",311,"As an agency user, I want a landing page to na..."
4461,309,"As a FABS user, I want to see correct status l...",312,"As an agency user, I want to submit my data el..."
4462,310,"As an agency user, I want to know when the sub...",311,"As an agency user, I want a landing page to na..."
4463,310,"As an agency user, I want to know when the sub...",312,"As an agency user, I want to submit my data el..."


### Process the User Stories

In [101]:
results_g02: list = []
exceptions_g02: list = []

process_user_stories(df_g02_pairs, results_g02, exceptions_g02)

NameError: name 'templat_request_two_user_stories' is not defined

### Store the results persistently

In [ ]:
results_collection: dict = {}
results_collection['#G02#'] = results_g02
if len(exceptions_g02) != 0: results_collection['#G02#Exceptions'] = exceptions_g02

save_to_json_persistent("benefit", results_collection)

## Data processing for: G03 loudoun

In [19]:
id_g03: str = "#G03#"
data_g03: list = datasets[id_g03]
df_g03: pd.DataFrame = transform_data_id_text(data_g03)
df_g03_pairs = transform_pairwise(df_g03)

In [18]:
df_g03.head()

,ID,User Story
0,315,"As a Public User, I want to Search for Informa..."
1,316,"As a ProspectiveApplicant, I want to research ..."
2,317,"As an Applicant, I want to Request PreApplicat..."
3,318,"As a Customer, I want to Create a Customer Por..."
4,319,"As an Applicant, I want to Submit Application,..."


In [22]:
display(df_g03_pairs)

,First ID,First User Story,Second ID,Second User Story
0,315,"As a Public User, I want to Search for Informa...",316,"As a ProspectiveApplicant, I want to research ..."
1,315,"As a Public User, I want to Search for Informa...",317,"As an Applicant, I want to Request PreApplicat..."
2,315,"As a Public User, I want to Search for Informa...",318,"As a Customer, I want to Create a Customer Por..."
3,315,"As a Public User, I want to Search for Informa...",319,"As an Applicant, I want to Submit Application,..."
4,315,"As a Public User, I want to Search for Informa...",320,"As an Applicant, I want to Submit Supporting D..."
...,...,...,...,...
1591,369,"As a County Staff Member, I want to create a c...",371,"As a Technical Staff member, I want to update ..."
1592,369,"As a County Staff Member, I want to create a c...",372,"As a Staff member, I want to Send or Post Cita..."
1593,370,"As a Staff member, I want to create each condo...",371,"As a Technical Staff member, I want to update ..."
1594,370,"As a Staff member, I want to create each condo...",372,"As a Staff member, I want to Send or Post Cita..."


### Process the User Stories

In [20]:
results_g03: list = None
exceptions_g03: list = None

if not THREADING:
    results_g03: list = []
    exceptions_g03: list = []

    process_user_stories(df_g03_pairs, results_g03, exceptions_g03)
else:
    process_requests_parallel(df_g03_pairs, f"redundancy-model-{MODEL_VERSION_NAME}", '#G03#')

### Store the results persistently

In [18]:
if not THREADING:
    results_collection: dict = {}
    results_collection['#G03#'] = results_g03
    if len(exceptions_g03) != 0: results_collection['#G03#Exceptions'] = exceptions_g03

    save_to_json_persistent(f"redundancy-model-{MODEL_VERSION_NAME}", results_collection)

## Data processing for: G04 recycling

In [135]:
id_g04: str = "#G04#"
data_g04: list = datasets[id_g04]
df_g04: pd.DataFrame = transform_data_id_text(data_g04)
df_g04_pairs = transform_pairwise(df_g04)

In [136]:
df_g04.head()

,ID,User Story
0,164,"As a user, I want to click on the address, so ..."
1,165,"As a user, I want to be able to anonymously vi..."
2,166,"As a user, I want to be able to enter my zip c..."
3,167,"As a user, I want to be able to get the hours ..."
4,168,"As a user, I want to have a flexible pick up t..."


In [137]:
display(df_g04_pairs)

,First ID,First User Story,Second ID,Second User Story
0,164,"As a user, I want to click on the address, so ...",165,"As a user, I want to be able to anonymously vi..."
1,164,"As a user, I want to click on the address, so ...",166,"As a user, I want to be able to enter my zip c..."
2,164,"As a user, I want to click on the address, so ...",167,"As a user, I want to be able to get the hours ..."
3,164,"As a user, I want to click on the address, so ...",168,"As a user, I want to have a flexible pick up t..."
4,164,"As a user, I want to click on the address, so ...",169,"As a user, I want to be able to select differe..."
...,...,...,...,...
1270,211,"As a user, I want to be able to browse through...",213,"As a recyclingfacility representative, I want ..."
1271,211,"As a user, I want to be able to browse through...",214,"As a recyclingfacility, I want to be able to c..."
1272,212,"As a recyclingfacility representative, I want ...",213,"As a recyclingfacility representative, I want ..."
1273,212,"As a recyclingfacility representative, I want ...",214,"As a recyclingfacility, I want to be able to c..."


### Process the User Stories

In [138]:
results_g04: list = []
exceptions_g04: list = []

process_user_stories(df_g04_pairs, results_g04, exceptions_g04)

### Store the results persistently

In [140]:
results_collection: dict = {}
results_collection['#G04#'] = results_g04
if len(exceptions_g04) != 0: results_collection['#G04#Exceptions'] = exceptions_g04

save_to_json_persistent(f"redundancy-model-{MODEL_VERSION_NAME}", results_collection)